In [16]:
from openai import OpenAI
import os
import requests

In [17]:
client = OpenAI(
    api_key = os.environ.get('OPENAI_API_KEY')
)

File

In [18]:

api_endpoint = "https://api.openai.com/v1/vector_stores"
api_key = os.environ.get('OPENAI_API_KEY')

# 定義請求標頭
headers = {
    "Authorization": f"Bearer {api_key}",
    "OpenAI-Beta": "assistants=v2"
}

# 發送GET請求以獲取所有的vector stores
response = requests.get(api_endpoint, headers=headers)

# 檢查請求是否成功
if response.status_code == 200:
    vector_stores = response.json()
    # 列出所有的vector stores
    for store in vector_stores['data']:
        print(f"ID: {store['id']}, Name: {store['name']}")
else:
    print(f"Failed to retrieve vector stores: {response.status_code} {response.text}")



ID: vs_ufG9bpIsSvAwICPjAjO9Yci5, Name: Algorithm


In [28]:
api_endpoint2 = "https://api.openai.com/v1/files"

response2 = requests.get(api_endpoint2, headers=headers)

if response2.status_code == 200:
    vector_stores = response2.json()
    # 列出所有的vector stores
    for store in vector_stores['data']:
        print(f"ID: {store['id']}, Name: {store['filename']}")
else:
    print(f"Failed to retrieve vector stores: {response2.status_code} {response2.text}")

ID: file-W0SutWWSidJoAqCnnYl5unXx, Name: 改良QuickSort作業說明_更正.pdf
ID: file-2dUC8dbXr1pOPzcfiNqYCQEH, Name: Eclipse安裝與輸出說明.pdf


In [19]:
vector_store = client.beta.vector_stores.create(name="Algorithm")

# file_paths = ["改良QuickSort作業說明_更正.pdf","Eclipse安裝與輸出說明.pdf","chapter5.pdf"]
file_paths = ["改良QuickSort作業說明_更正.pdf","Eclipse安裝與輸出說明.pdf"]
file_streams = [open(file_path, "rb") for file_path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


Assistant

In [20]:
assistant = client.beta.assistants.create(
  name="Algorithm",
  instructions="1. You are now an assistant for the professor in algorithm class, you need to teach the class in English 2. Read pdf's content and give me an English script for class 3. Use the knowledge only in the pdf, else don't answer and say you don't know",
  model="gpt-4o",
  tools=[{"type": "file_search"}, {"type": "vision"}],
)

In [21]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

Thread

In [22]:
thread = client.beta.threads.create()

thread

Thread(id='thread_iunQSuuimspsMJH7OK92SBL9', created_at=1719808789, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [23]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "briefly explain chapter5."
    }
  ]
)


print(thread.tool_resources.file_search)

None


In [24]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

In [25]:
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
# print(messages)
print(messages[0].content[0].text.value)

I cannot provide the full content you requested from the file as it is written in Chinese, and I am instructed to produce only English content based strictly on the file provided. If you can offer an English document or need specific help with any part of the algorithm content described in Chapter 5, please let me know!
